<a href="https://www.kaggle.com/code/patrickleal/transformation-cleaning-viz?scriptVersionId=144801583" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Importing Libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import matplotlib for data visualization
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
stadium_details_raw = pd.read_csv('/kaggle/input/brazilian-stadiums-dataset/brazilian-stadiums-details.csv')
stadiums_raw = pd.read_csv('/kaggle/input/brazilian-stadiums-dataset/brazilian-stadiums.csv')

In [ ]:
stadium_details_raw.head()

In [ ]:
stadiums_raw.head()

# **Functions Used**

**Function to transform the Date column:**

In [ ]:
def month_name_to_number(dataframe, col_index):
    ''' This function transforms the complete date in Portuguese 
    and returns the equivalent in dd-mm-yyyy '''
    
    # col_index is the index of the columns i want
    df = pd.DataFrame(dataframe.iloc[:, col_index]) # 1- creates a local dataframe with the column we want
    
    # creates a dict
    mon_to_number = {
    'janeiro': '01',
    'fevereiro': '02',
    'março': '03',
    'abril': '04',
    'maio': '05',
    'junho': '06',
    'julho': '07',
    'agosto': '08',
    'setembro': '09',
    'outubro': '10',
    'novembro': '11',
    'dezembro': '12'
    }
    
    column_name = df.columns[0] # 2- copy the label of the column
    
    for mon, num in mon_to_number.items():
        df[column_name] = df[column_name].str.replace(mon, num)                 # 3- replace the month name with the corresponding number
        df[column_name] = df[column_name].str.replace(str.capitalize(mon), num) # 4- replace the capitalized month name with the corresponding number   
        df[column_name] = df[column_name].str.replace(' de ', '-').str.strip()  # 5- replace ' de ' for '-'
    
    dataframe[column_name] = df[column_name]                                    # 6- adiciona a coluna limpa no dataframe
    return dataframe

# **Data Transformation**

## **Stadiums Dataset**

In [ ]:
stadiums_raw.info()

### **What to do?**

1. Change the labels of the columns.
1. Change the data type of the 'capacidade' column.
1. Replace 'Gov.' with 'Governo'

**1) Changing the labels of the columns**

In [ ]:
stadiums_df = stadiums_raw.copy() # create a copy os the raw dataset

# renaming de columns labels
stadiums_df.rename(columns={'Estádio': 'Stadium_Name', 
                           'Localidade': 'Locality',
                           'Unidade federativa': 'Federative_Units',
                           'Proprietário': 'Owner',
                           'Capacidade': 'Capacity'}, inplace=True) 
stadiums_df.head()

**2) Changing the data type of the 'capacidade':**

In [ ]:
stadiums_df['Capacity'] = stadiums_df['Capacity'].str.replace(' ','') # remove empty space 
stadiums_df['Capacity'] = stadiums_df['Capacity'].str.replace('.','') # remove the '.'
stadiums_df['Capacity'] = stadiums_df['Capacity'].astype(int)         # change to int 

In [ ]:
stadiums_df.info()

**3) Replace 'Gov.' with 'Governo'**

In [ ]:
stadiums_df['Owner'] = stadiums_df['Owner'].str.replace('Gov.', 'Governo').str.strip()
stadiums_df.head()

In [ ]:
stadiums_df.info()

## **Stadium Details Dataset**

In [ ]:
stadium_details_raw.info()

### **What to do?**

1. Change the column labels.
1. Transform the Date columns.
1. Transform the 'Publico Recorde' column.


In [ ]:
details_stadium_df = stadium_details_raw.copy() # creating a copy of the raw dataset
details_stadium_df.head()

**1) Renaming the Columns Labels**

In [ ]:
# choosing the new labels
new_labels = {'Estadio': 'Stadium_Name',
              'link': 'Wiki_Page_Link',
              'Nome Oficial': 'Official_Name',
              'Apelido': 'Nicknames',
              'Data Inauguracao': 'Opening_Date',
              'Público recorde': 'Record_Attendance',
              'Data recorde': 'Record_Date',
              'Partida com mais público': 'Match_with_the_highest_attendance'}

# renaming the columns
details_stadium_df.rename(columns=new_labels, inplace=True) 
details_stadium_df.head()

**2) Transforming the columns 'Date'**

In [ ]:
details_stadium_df[['Opening_Date', 'Record_Date']].head(15)

In [ ]:
#creating a df with the column 'Opening_Date'
Op_date_df = pd.DataFrame(details_stadium_df['Opening_Date'].str.replace(r'\([^)]*\)', '', regex=True))   # 1- removing all '()' and everything inside

Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'\[[^\]]*\]', '', regex=True)        # 2- removing all '[]' and everything inside
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'\n.*', '', regex=True)              # 3- removing all '\n' and everything that comes after
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'e reinauguração.*', '', regex=True) # 4- removing all 'e reinauguração' and everything that comes after
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'Reinauguração.*', '', regex=True)   # 5- removing all 'Reinauguração' and everything that comes after
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace('Original:', '')                      # 6- removing all 'Original:'
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace('Inauguração 1º', '01')               # 7- removing all 'Inauguração 1º' and replacing for '01'
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'— .*', '', regex=True)              # 8- removing all '— ' and everything that comes after
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace('1º', '01')                           # 9- removing all '1º' and replacing for '01'


I know there's probably a better way to do this, but I couldn't figure it out.

I had to use chatGPT  for some Regex.

Now I need to obtain only the first occurrence where the complete date appears.

I know by reading the wiki page that this is the original opening date.

In [ ]:
concatenated_lines = []                        # 1- the list that will store the cleaned lines

for i in range(len(Op_date_df)):
    try:

        line = []                              # 2- the list  of the current line in the loop
        
                                               # 3- in this block i get the splited elements of the current line
        for ele in Op_date_df.iloc[i]:
            value = ele.split()
            line.append(value)
            break

        line = line[0][:5]                     # 4- save only the first 5 elements 
        concat_line = ' '.join(line)           # 5- Receives the line with the first 5 elements of each line concatenated
        concatenated_lines.append(concat_line) # 6- append the current line in the lists with the lines concatenated
    except Exception as error:
        concatenated_lines.append(np.nan)      # 7- append a nan if an error occurs
    
clean_Date = pd.Series(concatenated_lines)     # 8- create a Series with all cleaned lines

In [ ]:
Op_date_df['clean_OP_Date'] = clean_Date
Op_date_df['clean_OP_Date'].head(10)

Checking inconsistencies

In [ ]:
Op_date_df[['Opening_Date', 'clean_OP_Date']].loc[124:133]

Fixing two more lines

In [ ]:
# replacing with the correct date
Op_date_df['clean_OP_Date'] = Op_date_df['clean_OP_Date'].str.replace('198427', '1984')
Op_date_df['clean_OP_Date'] = Op_date_df['clean_OP_Date'].str.replace('19403', '1940')
Op_date_df.loc[124:133]

Now it is correct, time to add the new clean opening column to the main dataframe

In [ ]:
details_stadium_df['opening_date_cleaned'] = Op_date_df['clean_OP_Date']


In [ ]:
details_stadium_df[['Opening_Date', 'opening_date_cleaned']].head()

In [ ]:
details_stadium_df.iloc[:, 8].head()

In [ ]:
# Using the function to tranform the dates
month_name_to_number(details_stadium_df, 8)

In [ ]:
details_stadium_df[['Opening_Date','Record_Date', 'opening_date_cleaned']].head()

**Time to do the same with 'Record_Date' column**

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# creating a df with the 'Record_Date' column
rec_date_df = pd.DataFrame(details_stadium_df['Record_Date'])
rec_date_df['clean_rec_date'] = rec_date_df['Record_Date'].str.replace(r'\([^)]*\)', '', regex=True)   # 1- removing all '()' and everything inside

# Looking at the stadium's Wiki page, both dates had the same record attendance,
# so I decided to save the more recent one, which is November 2, 2010
rec_date_df['clean_rec_date'] = rec_date_df['clean_rec_date'].str.replace('29 de maio[3] e ', '')

rec_date_df['clean_rec_date'] = rec_date_df['clean_rec_date'].str.replace(r'\[[^\]]*\]', '', regex=True) # 2- removing all '[]' and everything inside
rec_date_df['clean_rec_date'] = rec_date_df['clean_rec_date'].str.replace('1º', '01')                    # 3- removing all '1º' and replacing for '01'

In [ ]:
details_stadium_df['record_date_cleaned'] = rec_date_df['clean_rec_date']
details_stadium_df[['Record_Date', 'record_date_cleaned']].head()

In [ ]:
# using the function to clean the date
month_name_to_number(details_stadium_df, 9)

details_stadium_df[['Opening_Date', 'Record_Date', 'opening_date_cleaned', 'record_date_cleaned']].head()

**3) Transforming the 'Publico Recorde' column**